In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec

# Data Generation for 3D case
np.random.seed(0)
n_samples = 300
true_mean1 = [0, 0, 0]
true_mean2 = [5, 5, 5]
true_cov1 = [[1, 0.7, 0.5], [0.7, 1, -0.4], [0.5, -0.4, 1]]
true_cov2 = [[1.5, -0.9, 0.8], [-0.9, 1.5, -0.3], [0.8, -0.3, 1.5]]
data = np.concatenate((np.random.multivariate_normal(true_mean1, true_cov1, int(0.4 * n_samples)),
                       np.random.multivariate_normal(true_mean2, true_cov2, int(0.6 * n_samples))))

# EM Algorithm Implementation for 3D case
mean1, mean2 = np.array([-15, -20, -30]), np.array([10, 10, 10])
cov1, cov2 = np.array([[10, 5, 0], [0, 20, 0], [0, 0, 20]]), np.array([[10, 0, 0], [0, 10, 0], [0, 0, 10]])
log_likelihood = []
means1 = [mean1]
means2 = [mean2]

max_iter = 20
for iteration in range(max_iter):
    # E-step
    resp1 = multivariate_normal.pdf(data, mean1, cov1)
    resp2 = multivariate_normal.pdf(data, mean2, cov2)
    gamma = resp1 / (resp1 + resp2)

    # M-step
    mean1 = np.dot(gamma, data) / np.sum(gamma)
    cov1 = (np.dot((gamma[:, np.newaxis] * (data - mean1)).T, (data - mean1))
            / np.sum(gamma))
    
    mean2 = np.dot((1 - gamma), data) / np.sum(1 - gamma)
    cov2 = (np.dot(((1 - gamma)[:, np.newaxis] * (data - mean2)).T, (data - mean2))
            / np.sum(1 - gamma))
    
    # Update means history
    means1.append(mean1)
    means2.append(mean2)
    
    # Compute log-likelihood
    ll_current = np.sum(np.log(resp1 * gamma + resp2 * (1 - gamma)))
    log_likelihood.append(ll_current)

    # Visualization in 3D
    fig = plt.figure(figsize=(10, 6), dpi=200)
    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 2])  # Define the ratio of heights here

    ax1 = fig.add_subplot(gs[0], projection='3d')
    ax2 = fig.add_subplot(gs[1])

    # Plot data in 3D
    ax1.scatter(data[:, 0], data[:, 1], data[:, 2], c=gamma, cmap='coolwarm', marker='o')
    ax1.scatter(mean1[0], mean1[1], mean1[2], c='red', marker='s', linewidths=0.5, edgecolors='white', s=30, label='Mean 1')
    ax1.scatter(mean2[0], mean2[1], mean2[2], c='blue', marker='s', linewidths=0.5, edgecolors='white', s=30, label='Mean 2')
    ax1.set_title(f'Iteration {iteration + 1}, Log-Likelihood: {ll_current:.3f}')
    ax1.legend()

    # Plot log-likelihood
    ax2.plot(log_likelihood, '-o', label='Log-Likelihood', markersize=5)
    ax2.legend()

    plt.tight_layout()
    plt.show()
